In [2]:
import cobra
from cobra.io import load_model

Loading a CoBRApy model

In [4]:
model = load_model('salmonella')
model

Name,iYS1720
Memory address,174497290
Number of metabolites,2436
Number of reactions,3357
Number of genes,1707
Number of groups,0
Objective expression,1.0*BIOMASS_iRR1083_1 - 1.0*BIOMASS_iRR1083_1_reverse_12350
Compartments,"cytosol, extracellular space, periplasm"


In [13]:
pcpp = model.reactions.get_by_id('4PCPpp')
pcpp

Reaction identifier,4PCPpp
Name,"Tetrapeptide L,D-carboxypeptidase (periplasm)"
Memory address,0x30ba02d90
Stoichiometry,"LalaDgluMdapDala_p + h2o_p --> LalaDgluMdap_p + ala__D_p L-alanine-D-glutamate-meso-2,6-diaminoheptanedioate-D-alanine + H2O H2O --> L-alanine-D-glutamate-meso-2,6-diaminoheptanedioate + D-Alanine"
GPR,
Lower bound,0.0
Upper bound,1000.0


adding and removing metabolites, then checking the mass balance

In [45]:
pcpp.add_metabolites({
    model.metabolites.get_by_id('acmalt_c') : -1
}) 

pcpp.reaction

'LalaDgluMdapDala_p + acmalt_c + h2o_p --> LalaDgluMdap_p + ala__D_p'

`check_mass_balance()` returns elements violating mass balance

In [46]:
pcpp.check_mass_balance()

{'C': -14.0, 'H': -24.0, 'O': -12.0}

In [47]:
pcpp.subtract_metabolites({
    model.metabolites.get_by_id('acmalt_c') : -1
})

In [48]:
pcpp.check_mass_balance()

{}

Building the same reaction via string:
- arrow updates reversibility
- reaction ids should match from in the model

In [53]:
pcpp.reaction = 'LalaDgluMdapDala_p + acmalt_c + h2o_p --> LalaDgluMdap_p + ala__D_p'

### Metabolites

In [56]:
d_atp = model.metabolites.get_by_id('datp_c')
d_atp

Metabolite identifier,datp_c
Name,DATP C10H12N5O12P3
Memory address,0x30b3139d0
Formula,C10H12N5O12P3
Compartment,c
In 8 reaction(s),"BIOMASS_Ec_iAF1260_core_59p81M, BIOMASS_iRR1083_1, DATPHs, BIOMASS_invivo, RNTR1c, NTPP5, BIOMASS_iRR1083_metals_1, NDPK8"


In [57]:
d_atp.charge

-4

In [59]:
len(d_atp.reactions) # givess `frozen_set` of all reactions where d_atp is being used

8

### Genes

`gene_reaction_rule` is a boolean representation of the gene requirements for this reaction to be active

In [62]:
pcpp.gene_reaction_rule

''

In [63]:
pcpp.genes

frozenset()

Knock out modeling

In [77]:
pcpp.gene_reaction_rule = '(karthik or sanjai)'
pcpp.genes

frozenset({<Gene karthik at 0x30cb40090>, <Gene sanjai at 0x30ca1a310>})

In [74]:
model.genes.get_by_id('karthik')

Gene identifier,karthik
Name,
Memory address,0x30cb40090
Functional,True
In 1 reaction(s),4PCPpp


In [79]:
import cobra.manipulation


cobra.manipulation.knock_out_model_genes(
    model,
    ['karthik']
)
print('karthik KO:', pcpp.bounds)

cobra.manipulation.knock_out_model_genes(
    model,
    ['sanjai']
)
print('sanjai KO:', pcpp.bounds)

karthik KO: (0, 1000.0)
sanjai KO: (0, 0)


Sequential knocking out and studying the impact on Objective function without altering the original model

In [ ]:
model = load_model('salmonella')
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

## Building model